In [2]:
key_vault_name = 'kv_to-be-replaced'

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 4, Finished, Available)

In [19]:
# key_vault_name = 'ckmv2-keyvault' #'kv_to-be-replaced'

StatementMeta(, 423a745c-c808-41e8-b3e5-84a8a0a98d98, 21, Finished, Available)

In [ ]:
# select date_format(to_timestamp(timestamp,'yyyy/MM/dd HH:mm:ss'),"yyyy-MM-dd HH:mm:ss") as timeStamp from event

In [3]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 5, Finished, Available)

DataFrame[key: string, value: string]

In [3]:
# %%sql
# select StartTime, date(StartTime), date(date_format(to_timestamp(StartTime,'MM/dd/yyyy h:mm:ss a'),"yyyy-MM-dd HH:mm:ss")) from ckm_conv_metadata limit 2

StatementMeta(, 51023f4a-960b-4f4d-b2ca-c31d8bece9c5, 5, Finished, Available)

<Spark SQL result set with 2 rows and 3 fields>

In [4]:
df = spark.sql('''select b.conversation_id as ConversationId , date(date_format(to_timestamp(m.StartTime,'MM/dd/yyyy h:mm:ss a'),"yyyy-MM-dd HH:mm:ss")) as ConversationDate,
m.StartTime as StartTime, m.EndTime as EndTime, m.Duration AS Duration, m.CallerId as CallerId ,
m.CallReason as CallReason,m.ResolutionStatus as ResolutionStatus, 
m.AgentId as AgentId, m.AgentName as AgentName, m.Team as Team,
Merged_content,Merged_content_user,Merged_content_agent
from
(
    select conversation_id, concat_ws(' ', collect_list(Merged_content)) as Merged_content,
    concat_ws(' ', collect_list(Merged_content_user)) as Merged_content_user,
    concat_ws(' ', collect_list(Merged_content_agent)) as Merged_content_agent 
    from 
    (
        select conversation_id, row_id,DisplayText as Merged_content,
        case when SpeakerId = 'Guest-1' then DisplayText else '' end as Merged_content_agent,
        case when SpeakerId = 'Guest-2' then DisplayText else '' end as Merged_content_user
        from ckm_conv_messages order by conversation_id, row_id asc
    )
    group by conversation_id
) as b
inner join ckm_conv_metadata as m on b.conversation_id = m.ConversationId''')
# display(df)

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 93507974-4857-4872-9925-b654ac38dcd9)

In [5]:
from trident_token_library_wrapper import PyTridentTokenLibrary as tl

def get_secrets_from_kv(kv_name, secret_name):

    access_token = mssparkutils.credentials.getToken("keyvault")
    kv_endpoint = f'https://{kv_name}.vault.azure.net/'
    return(tl.get_secret_with_token(kv_endpoint,secret_name,access_token))

openai_api_type = "azure"
openai_api_version  = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-VERSION")
openai_api_base = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-ENDPOINT")
openai_api_key = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-KEY")

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 7, Finished, Available)

In [6]:
import os
import openai
import json
import time
import ast

def get_details(input_text):
    time.sleep(4)

    openai.api_type = openai_api_type
    openai.api_version = openai_api_version
    openai.api_base = openai_api_base
    openai.api_key =  openai_api_key

    # Construct the prompt  
    prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
            Summarize the conversation, key: summary .
            Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
            Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
            Identify the origin city of travel,key: OriginCity . 
            Identify the destination city of travel,key : DestinationCity . 
            Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
            Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
            Identify the name of hotel that was mentioned,key: Hotel . 
            Identify the name of airline if mentioned,key: Airline . 
            Identify the name of the agent,key: AgentName .
            Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
            Identify the main topic, key: topic .
            Identify the language of the text using ISO 639 two letter language identifier, key: lang .
            Identify input_text translated to english, return the same text if already in english, key: translated_text .
            Answer in JSON machine-readable format, using the keys from above. 
            Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

    max_retries = 5
    attempts = 0

    while attempts < max_retries:
        try:
            response = openai.ChatCompletion.create(
                engine= "gpt-35-turbo-16k", 
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": input_text}
                ]
            )

            result = ast.literal_eval(response['choices'][0]['message']['content'])
            if 'summary' in result and result['summary']:
                return result
            else:
                attempts += 1
                print(f"Attempt {attempts} failed. 'summary' not found in result. Trying again.")
                time.sleep(40)
        except Exception as e:
            attempts += 1
            print(f"Attempt {attempts} failed with error: {e}. Trying again.")
            time.sleep(40)

    print("Maximum number of retries reached. Exiting.")
    return {
        'summary': '',
        'satisfied': '',
        'avgSentiment': '',
        'OriginCity': '',
        'DestinationCity': '',
        'Complaint': '',
        'Compliment': "",
        'Hotel': '',
        'Airline': '',
        'AgentName': '',
        'keyPhrases': '',
        'topic': '',
        'lang': '',
        'translated_text': ''
    }


# input_str = '''Thank you for reaching out to the travel agency contact center. My name is Sarah Thompson. How may I assist you today?Hi Sarah, my name is Lisa Johnson. I recently traveled from Chicago to London and I had a terrible experience with the airline and hotel. I'm really frustrated with the service I received.I'm sorry to hear that, Lisa. Can you please provide me with some details about your trip so I can better understand the situation?Sure. I flew with United Airlines from Chicago to London, and I stayed at the Park Plaza Westminster Bridge hotel in London. I encountered issues with both during my trip.I apologize for any inconvenience you experienced. Could you please explain the specific problems you encountered with United Airlines?Absolutely. Firstly, the flight was delayed for more than three hours without any proper explanation. This caused a lot of inconvenience as I had connecting flights and had to reschedule my entire itinerary. Secondly, the onboard service was subpar. The flight attendants seemed disinterested and were not attentive to the passengers' needs.I understand how frustrating these situations can be, Lisa. I apologize for the lack of communication and the inconvenience caused by the delay. Delayed flights can be quite disruptive. Regarding the onboard service, I apologize for the unprofessional behavior of the flight attendants. I will make a note of your concerns and forward them to the airline for review.Thank you, Sarah. I appreciate your understanding. Now, regarding my hotel stay at the Park Plaza Westminster Bridge, the room was not up to standard. It was not properly cleaned, and there were maintenance issues with the bathroom.I apologize for the hotel's shortcomings, Lisa. It can be disappointing when accommodations don't meet expectations. I will contact the hotel management to address the cleanliness and maintenance issues you faced. In the meantime, is there anything specific you would like me to convey to the hotel?I would like them to know that I expect better cleanliness and maintenance in their rooms. It was really disappointing, especially considering the hotel's reputation.I completely understand, Lisa. I will communicate your concerns to the hotel management and emphasize the importance of ensuring a high level of cleanliness and maintenance throughout their property.Thank you, Sarah. I appreciate your assistance. Is there anything else you can do to help resolve these issues?Absolutely, Lisa. To further assist you, I will contact United Airlines to see if they can offer any compensation for the delay and address your concerns about the onboard service. Additionally, I will follow up with the Park Plaza Westminster Bridge hotel to ensure they take appropriate action regarding the cleanliness and maintenance issues in your room. I will keep you updated throughout the process.That sounds good, Sarah. I appreciate your efforts in resolving these matters. Thank you for your assistance.You're most welcome, Lisa. It is our priority to ensure that our customers have a pleasant travel experience. I will work diligently to resolve these issues for you. If you have any other questions or concerns, please don't hesitate to contact me.Thank you, Sarah. I will definitely reach out if I need any further assistance.'''
# res = get_details(input_str)

from pyspark.sql import Row

from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([
             StructField("summary", StringType(), True),
             StructField("satisfied", StringType(), True),
             StructField("avgSentiment", StringType(), True),
             StructField("OriginCity", StringType(), True),
             StructField("DestinationCity", StringType(), True),
             StructField("Complaint", StringType(), True),
             StructField("Compliment", StringType(), True),
             StructField("Hotel", StringType(), True),
             StructField("Airline", StringType(), True),
             StructField("AgentName", StringType(), True),
             StructField("keyPhrases", StringType(), True),
             StructField("topic", StringType(), True),
             StructField("lang", StringType(), True),
             StructField("translated_text", StringType(), True)
         ])

get_detail_udf = udf(lambda content: get_details(content),returnType=schema)

df_processed = df.select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason",
    "CallerID","Merged_content","Merged_content_agent","Merged_content_user"]) \
                .withColumn("Details", get_detail_udf(col("Merged_content"))) \
                .select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason","CallerID","Merged_content","Merged_content_agent","Merged_content_user", \
                          col("Details.summary").alias("summary"), \
                          col("Details.satisfied").alias("satisfied"), \
                          col("Details.avgSentiment").alias("avgSentiment"), \
                          col("Details.OriginCity").alias("OriginCity"), \
                          col("Details.DestinationCity").alias("DestinationCity"), \
                          col("Details.Complaint").alias("Complaint"), \
                          col("Details.Compliment").alias("Compliment"), \
                          col("Details.Hotel").alias("Hotel"), \
                          col("Details.Airline").alias("Airline"), \
                          # col("Details.AgentName").alias("AgentName"), \
                          col("Details.keyPhrases").alias("keyPhrases"), \
                          col("Details.topic").alias("topic"), \
                          col("Details.lang").alias("lang"), \
                          col("Details.translated_text").alias("translated_text")]) 
# display(df_processed)

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, cac09daa-8644-4a8e-b6fe-308b2b4aef10)

In [7]:
# spark.sql('drop table if exists ckm_conv_processed_audio_raw')

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 9, Finished, Available)

DataFrame[]

In [8]:
df_processed.write.format('delta').mode('append').option("overwriteSchema", "true").saveAsTable('ckm_conv_processed_raw')

StatementMeta(, f69d155d-4234-47a3-80cd-3d7bc48c2eae, 10, Finished, Available)

In [8]:
# Check the distinct values and the format of the StartTime field
# start_time_df = spark.sql("SELECT DISTINCT StartTime FROM ckm_conv_metadata")
# start_time_df.show()


StatementMeta(, 69804f08-375e-4e15-b9b0-dfc763f2d470, 10, Finished, Available)

+--------------------+
|           StartTime|
+--------------------+
|4/13/2024 7:48:40 AM|
|4/17/2024 5:49:25 PM|
|3/22/2024 12:49:3...|
| 2/7/2024 1:48:58 PM|
|4/11/2024 7:49:13 AM|
+--------------------+

